In [26]:
import numpy as np, tflearn, sys, tensorflow as tf
from tensorflow.contrib.keras.python.keras.utils import np_utils

filename = "wonderland.txt"
raw_text = open(filename,'r',encoding = 'utf-8').read()
raw_text = raw_text.lower()
chars = sorted(list(set(raw_text)))
print(len(chars))

56


In [ ]:
char2int = dict((char,ints) for ints, char in enumerate(chars))
int2char = dict((ints,char) for ints, char in enumerate(chars))
filename = 'mine'
seqlen = 100
lstmhid=320
keeprate = 0.80
train = []
true = []
tf.reset_default_graph()
for i in range(0, len(raw_text)-seqlen, 1):
    train.append([char2int[char] for char in raw_text[i:i+seqlen]])
    true.append(char2int[raw_text[i+seqlen]])
X = np.reshape(train,(len(train),seqlen,1))/float(len(chars))
y = np_utils.to_categorical(true)

In [ ]:
net = tflearn.input_data(shape=(None,X.shape[1],X.shape[2]))
net = tflearn.lstm(net,lstmhid)
net = tflearn.fully_connected(net,y.shape[1],activation='softmax')
net = tflearn.regression(net,optimizer='adam',learning_rate=0.005,loss='categorical_crossentropy')
model = tflearn.DNN(net,checkpoint_path = 'checkpoints/model.tfl.ckpt')
model.fit(X,y,snapshot_epoch=True,snapshot_step=5000,n_epoch=20,batch_size=128)
model.save(filename)

Training Step: 840  | total loss: 2.72418 | time: 554.779s
| Adam | epoch: 002 | loss: 2.72418 -- iter: 47104/60378


In [ ]:
model.load(filename)
for _ in range(5):
    p = train[np.random.randint(0,len(train)-1)]
    print("Seed:")
    print("\"",''.join([int2char[value] for value in p]), "\"")
    print("\n============================\n")
    for _ in range(100):
        tmp = np.argmax(model.predict((np.reshape(p, (1, len(p), 1))/float(len(chars)))))
        sys.stdout.write((int2char[tmp]))
        p.append(tmp)
        p = p[1:len(p)]
    print("\n------------------------------")